In [1]:
import "github.com/umbralcalc/stochadex/pkg/simulator"

%%
params := simulator.NewParams(make(map[string][]float64))
params.Set("chicken", []float64{1.0, 2.0})

In [2]:
import (
	"fmt"
	"log"
	"os"
	"os/exec"
	"strings"
	"time"

	"github.com/go-gota/gota/dataframe"
	"github.com/go-echarts/go-echarts/v2/charts"
	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/types"
    gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

func plotSomething() {
	csvStr := `
Country,Date,Age,Amount,Id
"United States",2012-02-01,50,112.1,01234
"United States",2012-02-02,32,321.31,54320
"United Kingdom",2012-02-03,17,18.2,12345
"United States",2012-02-04,32,321.31,54320
"United Kingdom",2012-02-05,NA,18.2,12345
"United States",2012-02-06,32,321.31,54320
"United States",2012-02-07,32,321.31,54320
"Spain",2012-02-08,66,555.42,00241
`

	// Load CSV data into dataframe
	df := dataframe.ReadCSV(strings.NewReader(csvStr))

	// Extract Date and Amount columns
	dateCol := df.Col("Date").Records()
	amountCol := df.Col("Amount").Float()

	// Prepare data for scatter plot
	var scatterData []opts.ScatterData
	for i, dateStr := range dateCol {
		date, err := time.Parse("2006-01-02", dateStr)
		if err != nil {
			log.Fatalf("error parsing date: %v", err)
		}
		scatterData = append(scatterData, opts.ScatterData{
			Value: []interface{}{date.Format("2006-01-02"), amountCol[i]},
		})
	}

	// Create scatter plot
	scatter := charts.NewScatter()
	scatter.SetGlobalOptions(
		charts.WithTitleOpts(opts.Title{Title: "Date vs Amount Scatter Plot"}),
		charts.WithXAxisOpts(opts.XAxis{
			Type: "category",
			Name: "Date",
			AxisLabel: &opts.AxisLabel{
				Rotate: 45, // Rotate labels to fit better
			},
		}),
		charts.WithYAxisOpts(opts.YAxis{
			Name: "Amount",
		}),
	)

	// Add data to scatter plot
	scatter.AddSeries("Amount", scatterData)//.
		// SetSeriesOptions(
		// 	charts.WithMarkPointNameTypeItemOpts(opts.MarkPointNameTypeItem{Name: "Max", Type: "max"}),
		// 	charts.WithMarkPointNameTypeItemOpts(opts.MarkPointNameTypeItem{Name: "Min", Type: "min"}),
		// )

	err := gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")
	if err != nil {
		fmt.Printf("Error: %+v\n", err)
	}
}

%%
plotSomething()